### **Automatic generation of responses using an LLM**

* **Summary:** This notebook is used for generating the responses: response_yes and response_no given the posts from the user. These responses are needed to train the LLM model using Direct Preference Optimization (DPO) approach. The LLM used is model="gemini-pro" from Google.
* **Datasets:** addiction_2018_features_tfidf_256.csv from https://zenodo.org/records/3941387

In [10]:
! pip install langchain

! pip install --upgrade --quiet  langchain-google-genai

In [11]:
from langchain_google_genai import GoogleGenerativeAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain

import pandas as pd

### Fetch Google API Key
https://makersuite.google.com is the link for generating the API key.

In [57]:
# Used to securely store your API key
from google.colab import userdata

GOOGLE_API_KEY=userdata.get('GOOGLE_API_KEY')

In [67]:
def detect_condition(question):

    prompt_template = PromptTemplate(
        input_variables = ['question'],
        template = ("""
            Provide me two responses for the {question}: "yes_response" and "no_response". Use consistent formatting for the labels.
            The yes_response will be the correct evaluation provided as a mental health advisor.
            The no_response will be an incorrect evaluation of the query.
            While providing yes_response, I want you to give the best personal advice based on {question}.
            In the yes_response, ask them to contact their nearest helpline. Don't provide contact number since you don't know the location of the user.
        """)
    )

    llm = GoogleGenerativeAI(model="gemini-pro", google_api_key=GOOGLE_API_KEY, temperature=0.2)
    chain = LLMChain(llm=llm, prompt=prompt_template)

    response = chain.run({'question': question})

    return response

### Reading The Metal Health Data

In [95]:
df = pd.read_csv("./source/mental_health.csv")

df.head()

,subreddit,author,date,post,automated_readability_index,coleman_liau_index,flesch_kincaid_grade_level,flesch_reading_ease,gulpease_index,gunning_fog_index,...,tfidf_wish,tfidf_without,tfidf_wonder,tfidf_work,tfidf_worri,tfidf_wors,tfidf_would,tfidf_wrong,tfidf_x200b,tfidf_year
0,addiction,SearchSmegmaongoogle,2018/01/01,Deciding to go of tramadol Well after never ta...,5.651264,5.245836,6.314583,81.454306,67.250000,8.861111,...,0.299199,0.000000,0.0,0.000000,0.0,0.132575,0.000000,0.0,0.0,0.000000
1,addiction,420somthing,2018/01/02,My vyvanse addiction... It has gotten pretty b...,0.061000,2.401664,2.085000,93.472500,91.333333,4.333333,...,0.000000,0.248395,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.000000
2,addiction,PurposedPorpoise,2018/01/02,Quitting coke and nicotine I'm gonna start by ...,3.195416,4.491779,5.042790,82.295026,73.503311,8.960783,...,0.000000,0.099761,0.0,0.075342,0.0,0.000000,0.073517,0.0,0.0,0.133289
3,addiction,iamauserunknown,2018/01/02,Is it OK to leave a drug addict you love? Man...,8.166136,7.535312,8.157072,71.057611,62.210332,11.801898,...,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.141137
4,addiction,whattheheyyy,2018/01/02,My brother has a problem I'm not against weed....,1.473585,3.190691,2.855935,94.388683,79.158730,6.140181,...,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.000000


In [96]:
df = pd.DataFrame(df["post"])

# Test the first 10 rows
df = df.iloc[0:10]
df = df.reset_index(drop=True)

df

,post
0,Deciding to go of tramadol Well after never ta...
1,My vyvanse addiction... It has gotten pretty b...
2,Quitting coke and nicotine I'm gonna start by ...
3,Is it OK to leave a drug addict you love? Man...
4,My brother has a problem I'm not against weed....
5,I have a buddy and he is hitting the methadone...
6,Nasha Mukti Kendra in Delhi for Drugs Alcohal ...
7,Alcoholism + E.D. + trich anyone have a histor...
8,My New Years resolution: to kick the porn habi...
9,Can I “kidnap” a drug addict if it is for thei...


### Generate Response

In [97]:
all_responses = []

for post in df["post"]:
    response = detect_condition(post)
    all_responses.append(response)

all_responses

['',
 "**yes_response:**\n\nVyvanse addiction can be a very difficult thing to deal with. It's important to remember that you're not alone, and that there are people who can help you. If you're struggling with Vyvanse addiction, please reach out to a mental health professional or contact your nearest helpline. There is help available, and you don't have to go through this alone.\n\n**no_response:**\n\nVyvanse addiction is not a serious problem. You can just stop taking it and you'll be fine.",
 "**yes_response:**\n\nCongratulations on your progress! It's great that you've been able to stay away from cocaine for over 10 weeks. That's a significant accomplishment. It's also understandable that you're struggling with nicotine withdrawal. Nicotine is a highly addictive substance, and it can be difficult to quit. However, it's important to remember that you're not alone. There are many resources available to help you quit smoking. I would encourage you to contact your nearest helpline. They

In [102]:
import re

# Initialize lists to store responses
yes_responses = []
no_responses = []

# Define a regular expression pattern to capture labels
label_pattern = r"\n{0,2}(?:\*\*(?:no_response|No_response|yes_response|Yes_response)(?:\*\*|:))\s*\n{0,2}"

# Iterate through the data
for item in all_responses:
    # Split the item into responses based on label pattern
    responses = re.split(label_pattern, item)
    # Remove empty strings from responses
    responses = [response.strip() for response in responses if response.strip()]
    # Assign responses to respective lists
    if len(responses) >= 2:
        yes_response = responses[0]
        # Replace variations of yes_response label
        yes_response = yes_response.replace("**yes_response**", "").replace("**yes_response**", "")
        yes_responses.append(yes_response)
        no_responses.append('\n'.join(responses[1:]))
    else:
        yes_response = None
        no_response = None
        yes_responses.append(yes_response)
        no_responses.append(no_response)


# Print or use yes_response_list and no_responselist as needed
for yes_response, no_response in zip(yes_responses, no_responses):
    print("yes_response:", yes_response)
    print("no_responses:", no_response)


yes_response: None
no_responses: None
yes_response: **

Vyvanse addiction can be a very difficult thing to deal with. It's important to remember that you're not alone, and that there are people who can help you. If you're struggling with Vyvanse addiction, please reach out to a mental health professional or contact your nearest helpline. There is help available, and you don't have to go through this alone.
no_responses: **

Vyvanse addiction is not a serious problem. You can just stop taking it and you'll be fine.
yes_response: **

Congratulations on your progress! It's great that you've been able to stay away from cocaine for over 10 weeks. That's a significant accomplishment. It's also understandable that you're struggling with nicotine withdrawal. Nicotine is a highly addictive substance, and it can be difficult to quit. However, it's important to remember that you're not alone. There are many resources available to help you quit smoking. I would encourage you to contact your neares

In [103]:
# Create a DataFrame
responses_df = pd.DataFrame({'yes_response': yes_responses, 'no_response': no_responses})
responses_df['yes_response'] = responses_df['yes_response'].str.replace(r"\*\*\s*\n{0,2}", "", regex=True)
responses_df['no_response'] = responses_df['no_response'].str.replace(r"\*\*\s*\n{0,2}", "", regex=True)

responses_df

,yes_response,no_response
0,None,None
1,Vyvanse addiction can be a very difficult thin...,Vyvanse addiction is not a serious problem. Yo...
2,Congratulations on your progress! It's great t...,It's great that you've been able to stay away ...
3,It's understandable that you're feeling torn a...,You should stay with your girlfriend no matter...
4,It's understandable that you're concerned abou...,It's not a big deal. Your brother is just expe...
5,I understand that you're feeling sorry for you...,I don't think it's fair to assume that your fr...
6,I understand that you're struggling with drug ...,"I'm sorry, but I'm not able to provide you wit..."
7,It's understandable that you're feeling overwh...,It's not uncommon for people to struggle with ...
8,I understand that you're struggling with a por...,"I'm sorry, but I'm not qualified to give you a..."
9,None,None


In [104]:
llm_df = pd.concat([df, responses_df],axis=1)
# llm_df.to_csv("./output/responses.csv")

llm_df

,post,yes_response,no_response
0,Deciding to go of tramadol Well after never ta...,None,None
1,My vyvanse addiction... It has gotten pretty b...,Vyvanse addiction can be a very difficult thin...,Vyvanse addiction is not a serious problem. Yo...
2,Quitting coke and nicotine I'm gonna start by ...,Congratulations on your progress! It's great t...,It's great that you've been able to stay away ...
3,Is it OK to leave a drug addict you love? Man...,It's understandable that you're feeling torn a...,You should stay with your girlfriend no matter...
4,My brother has a problem I'm not against weed....,It's understandable that you're concerned abou...,It's not a big deal. Your brother is just expe...
5,I have a buddy and he is hitting the methadone...,I understand that you're feeling sorry for you...,I don't think it's fair to assume that your fr...
6,Nasha Mukti Kendra in Delhi for Drugs Alcohal ...,I understand that you're struggling with drug ...,"I'm sorry, but I'm not able to provide you wit..."
7,Alcoholism + E.D. + trich anyone have a histor...,It's understandable that you're feeling overwh...,It's not uncommon for people to struggle with ...
8,My New Years resolution: to kick the porn habi...,I understand that you're struggling with a por...,"I'm sorry, but I'm not qualified to give you a..."
9,Can I “kidnap” a drug addict if it is for thei...,None,None
